# Zajjal terhelt dinamikai rendszer állapotbecslése szintén zajos mérési adatokból

## A Kálmán szűrő (lineáris rendszerek)

## Részecskeszűrők (nemlineáris rendszerek)

# Dinamikai rendszerek identifikációja

## ARX modellek

## LSTM neurális hálók

# További olvasnivalók

1. https://en.wikipedia.org/wiki/Delay_differential_equation
2. https://arxiv.org/pdf/2001.07392.pdf 
3. https://www.sciencedirect.com/bookseries/mathematics-in-science-and-engineering/vol/191/suppl/C 
4. https://www.sciencedirect.com/science/article/abs/pii/S0168927405000929